# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5].values

array([0, 0, 0, 1, 0])

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [10]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer_1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer_2 = 3

# Define the number of neurons in the output layer
number_output_neurons = 1

In [11]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(Dense(units=hidden_nodes_layer_1, input_dim=input_nodes, activation="relu"))

# Add the second hidden layer
nn_model.add(Dense(units=hidden_nodes_layer_2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [12]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 0s 758us/step - loss: 0.5115 - accuracy: 0.7540
Epoch 2/50
38/38 [==============================] - 0s 449us/step - loss: 0.5114 - accuracy: 0.7515
Epoch 3/50
38/38 [==============================] - 0s 376us/step - loss: 0.5104 - accuracy: 0.7523
Epoch 4/50
38/38 [==============================] - 0s 347us/step - loss: 0.5100 - accuracy: 0.7515
Epoch 5/50
38/38 [==============================] - 0s 337us/step - loss: 0.5092 - accuracy: 0.7515
Epoch 6/50
38/38 [==============================] - 0s 344us/step - loss: 0.5092 - accuracy: 0.7540
Epoch 7/50
38/38 [==============================] - 0s 331us/step - loss: 0.5082 - accuracy: 0.7531
Epoch 8/50
38/38 [==============================] - 0s 323us/step - loss: 0.5075 - accuracy: 0.7498
Epoch 9/50
38/38 [==============================] - 0s 323us/step - loss: 0.5070 - accuracy: 0.7523
Epoch 10/50
38/38 [==============================] - 0s 343us/step - loss: 0.5066 - accuracy: 0.7498

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [22]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)    

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5069 - accuracy: 0.7425 - 19ms/epoch - 1ms/step
Loss: 0.506858229637146, Accuracy: 0.7425000071525574


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [23]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [24]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_model_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [25]:
# Make predictions with the test data
predictions = nn_model_imported.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predictions[:5]

13/13 - 0s - 31ms/epoch - 2ms/step


array([[0.31501278],
       [0.2444051 ],
       [0.8703169 ],
       [0.6625731 ],
       [0.93030655]], dtype=float32)

In [26]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_rounded = [round(predictions[0], 0) for predictions in predictions]
predictions_df = pd.DataFrame(predictions_rounded, columns=["predictions"])
predictions_df

,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [27]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_rounded))

              precision    recall  f1-score   support

           0       0.72      0.74      0.73       188
           1       0.77      0.74      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.74      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

Possible data to collect would be:
 * age of student - Age could influence desired loan options, ability to repay, terms requested, and probability that degree will be completed.
 * gpa - Grade point average may influence probability of degree completion, employability, etc. which in turn may impact what options are available to the student.
 * education level completed - Education level completed would show whether student has completed higher education and is looking to extend the degree. For example, a student have completed a bachelor's but is now working towards a master's degree. Also, high school versus GED might be a distinguishing factor in credit worthiness.
 * field of study - the chosen field may have a higher or lower level of employment post-education which would impact risk of defaulting thus impacting the loan options.
 * degree/certificate applying for - The degree could impact repayment. For example, a doctor or lawyer may have residency or supervisory period after graduation and/or may also require exams to be passed before able to earn expected pay scale. Whereas, some degrees may be immediately employable. This could impact desired terms. Some loan options are only available to undergraduates whereas other loan options are available to graduates. Some degrees are for non-traditional students and the repayment terms could be immediate while the degree is still in progress. 
 * college/university - This would impact the amount needed. A public university versus a private university effects financing needs.
 * current location - This would impact in state and out of state tuition along with financing needs.
 * employment status of student - Unemployed, part time employment, or full time employment will impact repayment schedules as well as amounts to finance. Some loans may be directed to tuition only while others cover living expenses.
 * employment status of parents - Parents of traditional students are often considered in loan options for students.
 * loan amount requested - Some loan options may have minimum or maximum requirements or lifetime limitations that are impacted.
 * loan purpose - Some loan options may be limited to specific purpose for example tuition only, living expenses, etc.
 * repayment terms - Some loan options may be impacted by repayment terms requested. For example, if student is willing to begin repayment immediately, certain interest rates may be available or special terms.
 * cosigners - Loan options may be increased if student has a cosigner willing to sign. For example, a student requesting $200,000 in loan amount but has a cosigner may have more options than a student with no cosigners.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

There are two types of filtering that would be applicable to the data listed in #1.

Collaborative filtering would suggest items to students based on the similarity to other users. So the data collected would compare students to other students and what loan options were selected. For example, traditional students between the ages of 17-23 going for a bachelor's degree, would be given loan options that others in that demographic have recommended/selected.

It would also be possible to create a context-based filtering recommendation system. For example, if a student was already employed but going for a graduate degree or certificate, then only select loan options would be available.

The data collected is flexible enough to develop either type of filtering process. Futhermore, hybrid modeling can often outperform modeling that is only focused on one type of system because it is more comprehensive in nature.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

One real world challenge would be race discrimination in the recommendation process. #1 excluded any requests for race/ethnicity in the data collected or used in the recommendation system. It would be important to not collect this data and have any bias introduced to the model related to race/ethnicity.

A second real world challenge would be sex or sexual orientation discrimination in the recommendation process. #1 excluded any requests for information related to sex or sexual orientation. It would be important to not collect this data and have any bias introduced to the model related to sex/sexual orientation.

The recommendation system should be free of legally protected fields/information. In addition to the two mentioned, no information should be collected about religious preferences/affiliations or disabilities. It is important to follow both federal and state laws in regards to education and finances. Since the recommendation system covers both education and finance sectors, it would be important to have full knowledge of laws, requirements, and documentation procedures in collecting the data and developing the models. There may be applicable reporting requirements under both or only one sector that would necessitate additional steps in the modeling process. If after reviewing the applicable laws, it may be necessary to remove certain data collected from the source, for example age. While age is currently listed as a feature to collect in the data, legal requirements may require the elimination of certain fields. It would be important to insure that the information collected met all requirements before modeling and recommending.

There may also be additional requirements for GDPR compliance. These requirements will also need to be implemented. There may also be some requirements around deletion of data if requested by the student and/or parents. The data collection and storage process will need to be reviewed to insure compliance with GDPR. If data is used in modeling, it may be necessary to remove requested data and then retrain models.

The data recommended in #1 excluded PII (Personally Identifiable Information). It would be worth investigating whether PII can be collected to use in the recommendation system. Some examples of PII would include student's annual income, credit score, and debt obligations and parents' annual income, credit score, and debt obligations. All of these data points could have valuable impact on the modeling results and the recommendation system for loan options. However, the legal requirements around handling of PII may be cost prohibitive, so it may be better to exclude these features. 
